# TP : Système de Recommandation avec un LSTM

### Etape 1 : Importation des données

#### 1. Téléchargement des données :
- Téléchargez le dataset "E-Commerce Dataset" depuis Kaggle en utilisant l’API kagglehub.
- Lien vers le dataset : [E-Commerce Dataset](https://www.kaggle.com/datasets/davidafolayan/e-commerce-dataset/data)

### Etape 2 : Chargement et exploration des données

#### 2. Charger les données :
- Chargez le fichier CSV contenant les données transactionnelles en utilisant pandas.
- Affichez un aperçu des colonnes disponibles et des premières lignes du dataset.


In [29]:
import pandas as pd

# Charger les données dans un DataFrame pandas
df = pd.read_csv(r"C:\Users\cated\Desktop\dev\IA school\Machine learning supervisé recommandation\Ecommerce_data.csv")
df.head()


,customer_id,customer_first_name,customer_last_name,category_name,product_name,customer_segment,customer_city,customer_state,customer_country,customer_region,...,order_date,order_id,ship_date,shipping_type,days_for_shipment_scheduled,days_for_shipment_real,order_item_discount,sales_per_order,order_quantity,profit_per_order
0,C_ID_45866,Mary,Fuller,Office Supplies,Xerox 1913,Corporate,New Rochelle,New York,United States,East,...,11/5/2022,O_ID_3001072,11/7/2022,Second Class,2,2,35.0,500.0,5,223.199997
1,C_ID_44932,Alan,Edelman,Office Supplies,#6 3/4 Gummed Flap White Envelopes,Corporate,Houston,Texas,United States,Central,...,20-06-2022,O_ID_3009170,23-06-2022,Second Class,2,3,85.0,500.0,5,199.199997
2,C_ID_70880,Mary,Gayman,Office Supplies,Belkin 8 Outlet Surge Protector,Consumer,Louisville,Kentucky,United States,South,...,25-06-2022,O_ID_3047567,30-06-2022,Standard Class,4,5,75.0,44.0,5,195.500000
3,C_ID_33157,Raymond,Eason,Office Supplies,GBC VeloBinder Manual Binding System,Corporate,Chicago,Illinois,United States,Central,...,10/6/2022,O_ID_3060575,10/10/2022,Second Class,2,4,60.0,254.0,1,220.000000
4,C_ID_58303,Mary,Gonzalez,Furniture,Eldon Pizzaz Desk Accessories,Home Office,Philadelphia,Pennsylvania,United States,East,...,2/5/2022,O_ID_3064311,8/1/2022,First Class,1,2,125.0,500.0,1,97.500000


### Etape 3 : Prétaitement des données

#### 3. Sélectionner les colonnes pertinentes :
- Gardez uniquement les colonnes suivantes : customer id, product name, et order date.

#### 4. Réduction de la taille du dataset :
- Réduisez la taille du dataset à 20 % pour simplifier l’exécution des modèles.

#### 5. Conversion des dates et tri :
- Convertissez la colonne order date en format datetime et triez les achats par customer id et order date.

#### 6. Agrégation des séquences :
- Regroupez les achats par client en créant une séquence unique de produits achetés.

In [80]:
# Étape 3 : Prétaitement des données
# Sélectionner les colonnes pertinentes
df_reduit = df[['customer_id', 'product_name', 'order_date']]

# Afficher les premières lignes du DataFrame réduit
print("\nDataFrame réduit :")
print(df_reduit.head())

# Réduction de la taille du dataset à 20%
df_reduit = df_reduit.sample(frac=0.20, random_state=1)
print(f"Nombre de lignes après réduction : {len(df_reduit)}")

# Conversion des dates en format datetime en gérant les différents formats
df_reduit['order_date'] = pd.to_datetime(df_reduit['order_date'], format="%d-%m-%Y", errors='coerce')
df_reduit['order_date'] = df_reduit['order_date'].fillna(pd.to_datetime(df_reduit['order_date'], format="%m/%d/%Y", errors='coerce'))

# Enlever les dates invalides
df_reduit = df_reduit.dropna(subset=['order_date'])

# Trier les achats par customer_id et order_date
df_reduit = df_reduit.sort_values(by=['customer_id', 'order_date'])
print(df_reduit.head())

# Agrégation des séquences
df_agg = df_reduit.groupby('customer_id')['product_name'].apply(lambda x: ', '.join(x)).reset_index()
df_agg.rename(columns={'product_name': 'products_sequence'}, inplace=True)
print(df_agg.head())




DataFrame réduit :
  customer_id                          product_name  order_date
0  C_ID_45866                            Xerox 1913   11/5/2022
1  C_ID_44932    #6 3/4 Gummed Flap White Envelopes  20-06-2022
2  C_ID_70880       Belkin 8 Outlet Surge Protector  25-06-2022
3  C_ID_33157  GBC VeloBinder Manual Binding System   10/6/2022
4  C_ID_58303         Eldon Pizzaz Desk Accessories    2/5/2022
Nombre de lignes après réduction : 22654
       customer_id                                       product_name  \
112094  C_ID_25005                             GBC Wire Binding Combs   
43163   C_ID_25006                     Cisco SPA525G2 5-Line IP Phone   
43165   C_ID_25006  Kensington 7 Outlet MasterPiece Power Center w...   
18488   C_ID_25008                                         Newell 318   
29860   C_ID_25011      Wilson Jones 1" Hanging DublLock Ring Binders   

       order_date  
112094 2022-08-24  
43163  2021-07-24  
43165  2021-08-20  
18488  2022-02-15  
29860  2022-10-2

### Etape 4 : Tokenisation et création des paires d’entrée-sortie

#### 7. Tokenisation des séquences :
- Utilisez un tokenizer pour convertir les noms de produits en indices numériques.

#### 8. Créer les paires entrée-sortie :
- Créez des paires X et y où X est une séquence partielle et y est le produit suivant dans la séquence.

#### 9. Remplir les séquences :
- Utilisez le padding pour uniformiser les longueurs des séquences.


In [66]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# 7. Tokenisation des séquences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_agg['products_sequence'])
sequences = tokenizer.texts_to_sequences(df_agg['products_sequence'])

# 8. Créer les paires entrée-sortie
X, y = [], []
for seq in sequences:
    for i in range(1, len(seq)):
        X.append(seq[:i])
        y.append(seq[i])

# Trouver la longueur maximale des séquences
max_sequence_length = max(len(seq) for seq in X)

# 9. Remplir les séquences
X = pad_sequences(X, maxlen=max_sequence_length, padding='pre')
y = np.array(y)


### Etape 5 : Création et entraînement du modèle LSTM

#### 10. Définir le modèle :
- Construisez un modèle LSTM avec les couches suivantes :
  - Une couche Embedding pour convertir les indices des produits en vecteurs de caractéristiques.
  - Une couche LSTM.
  - Deux couches Dense, dont la dernière avec une activation softmax.

### 11. Compiler et entraîner le modèle :
- Compilez le modèle avec une fonction de perte adaptée et entraînez-le.

In [70]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Définir les paramètres du modèle
vocab_size = len(tokenizer.word_index) + 1  # Taille du vocabulaire
embedding_dim = 50  # Dimension des embeddings
max_sequence_length = max([len(x) for x in X])  # Longueur maximale des séquences

# 10. Définir le modèle
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

# 11. Compiler et entraîner le modèle
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
history = model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/20


C:\Users\cated\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step - accuracy: 6.2500e-04 - loss: 5.9992 - val_accuracy: 0.0099 - val_loss: 6.0025
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.0072 - loss: 5.9895 - val_accuracy: 0.0099 - val_loss: 6.0208
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.0132 - loss: 5.9410 - val_accuracy: 0.0000e+00 - val_loss: 6.2588
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.0135 - loss: 5.7908 - val_accuracy: 0.0000e+00 - val_loss: 6.5318
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.0122 - loss: 5.6875 - val_accuracy: 0.0000e+00 - val_loss: 6.7070
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.0174 - loss: 5.6641 - val_accuracy: 0.0000e+00 - val_loss: 6.8708
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.0097 - loss: 5.6022 - val_accuracy: 0.0000e+00 - val_loss: 7.0638
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.0093 - loss: 5.5744 - val_accuracy: 0.0000e+00 - 

### Etape 6 : Prédiction

#### 12. Créer une fonction de prédiction :
- Implémentez une fonction qui prend une séquence de produits et retourne les prochains produits les plus probables.

#### 13. Tester les prédictions :
- Utilisez un exemple de séquence pour générer des prédictions.

In [ ]:
# 12. Créer une fonction de prédiction
def predict_next_products(model, tokenizer, input_sequence, num_predictions=1):
    """
    Prédire les prochains produits les plus probables.

    Args:
    model -- le modèle LSTM entraîné
    tokenizer -- le tokenizer utilisé pour convertir les noms de produits
    input_sequence -- la séquence d'entrée des produits
    num_predictions -- le nombre de prédictions à générer

    Retourne:
    Une liste des prochains produits les plus probables
    """
    for _ in range(num_predictions):
        # Convertir la séquence d'entrée en indices
        encoded_sequence = tokenizer.texts_to_sequences([input_sequence])[0]
        # Ajouter un padding pour correspondre à la longueur maximale de séquence
        padded_sequence = pad_sequences([encoded_sequence], maxlen=max_sequence_length, padding='pre')

        # Prédire le produit suivant
        predicted_index = model.predict(padded_sequence, verbose=0)
        predicted_index = np.argmax(predicted_index, axis=1)[0]

        # Convertir l'indice prédit en produit
        predicted_product = tokenizer.index_word[predicted_index]
        
        # Ajouter le produit prédit à la séquence d'entrée
        input_sequence += ' ' + predicted_product

    return input_sequence.split()[-num_predictions:]

# 13. Tester les prédictions
example_sequence = "product_1 product_2 product_3"
predicted_products = predict_next_products(model, tokenizer, example_sequence, num_predictions=3)
print(f"Produits prédits : {predicted_products}")


### Etape 7 : Questions de réflexion

#### 14. Améliorations possibles :
- Quelles approches pourriez-vous utiliser pour améliorer les performances du modèle ?
  - Augmenter la taille du dataset pour capturer plus de variations.
  - Utiliser des techniques de régularisation telles que la L2 regularization ou le dropout pour éviter le surapprentissage.
  - Expérimenter avec des architectures de modèles différentes, comme les GRU ou Transformer.
  - Fine-tuning des hyperparamètres tels que le taux d'apprentissage, la taille des couches, et le nombre de neurones.

- Proposez des métriques d’évaluation adaptées pour ce type de système de recommandation.
  - Précision@k : Pourcentage de fois où le produit correct est parmi les k premiers produits recommandés.
  - Recall@k : Pourcentage de fois où tous les produits corrects sont parmi les k premiers produits recommandés.
  - F1-Score : Harmonie entre la précision et le rappel pour évaluer le modèle.
  - AUC-ROC : Mesure la performance du modèle de classification à différents seuils de classification.
  - NDCG (Normalized Discounted Cumulative Gain) : Mesure l'efficacité des recommandations en prenant en compte la position des produits recommandés.

Essayez ce code pour implémenter les prédictions et utilisez les questions de réflexion pour améliorer et évaluer votre modèle. Si vous avez besoin d'aide supplémentaire ou d'autres modifications, n'hésitez pas à demander! 😊


In [1]:
# Importer les bibliothèques nécessaires
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GRU
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, ndcg_score

# Étape 2 : Charger et explorer les données
df = pd.read_csv(r"C:\Users\cated\Desktop\dev\IA school\Machine learning supervisé recommandation\Ecommerce_data.csv")
print("Aperçu des colonnes :")
print(df.columns)
print("\nPremières lignes du dataset :")
print(df.head())

# Étape 3 : Prétaitement des données
df_reduit = df[['customer_id', 'product_name', 'order_date']]
print("\nDataFrame réduit :")
print(df_reduit.head())

# Réduction de la taille du dataset à 20%
df_reduit = df_reduit.sample(frac=0.20, random_state=1)
print(f"Nombre de lignes après réduction : {len(df_reduit)}")

# Conversion des dates et tri
df_reduit['order_date'] = pd.to_datetime(df_reduit['order_date'], format="%d-%m-%Y", errors='coerce')
df_reduit['order_date'] = df_reduit['order_date'].fillna(pd.to_datetime(df_reduit['order_date'], format="%m/%d/%Y", errors='coerce'))

# Enlever les dates invalides
df_reduit = df_reduit.dropna(subset=['order_date'])

# Trier les achats par customer_id et order_date
df_reduit = df_reduit.sort_values(by=['customer_id', 'order_date'])
print(df_reduit.head())

# Agrégation des séquences
df_agg = df_reduit.groupby('customer_id')['product_name'].apply(lambda x: ', '.join(x)).reset_index()
df_agg.rename(columns={'product_name': 'products_sequence'}, inplace=True)
print(df_agg.head())

# Étape 4 : Tokenisation et création des paires d’entrée-sortie
# 7. Tokenisation des séquences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_agg['products_sequence'])
sequences = tokenizer.texts_to_sequences(df_agg['products_sequence'])

# 8. Créer les paires entrée-sortie
X, y = [], []
for seq in sequences:
    for i in range(1, len(seq)):
        X.append(seq[:i])
        y.append(seq[i])

# Trouver la longueur maximale des séquences
max_sequence_length = max(len(seq) for seq in X)

# 9. Remplir les séquences
X = pad_sequences(X, maxlen=max_sequence_length, padding='pre')
y = np.array(y)

# Étape 5 : Création et entraînement du modèle LSTM
# Définir les paramètres du modèle
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100  # Augmenter la dimension des embeddings
max_sequence_length = max([len(x) for x in X])

# Définir le modèle
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(128))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

# Compiler le modèle
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
history = model.fit(X, y, epochs=20, batch_size=64, validation_split=0.2)

# Étape 6 : Prédiction
# 12. Créer une fonction de prédiction
def predict_next_products(model, tokenizer, input_sequence, num_predictions=1):
    """
    Prédire les prochains produits les plus probables.

    Args:
    model -- le modèle LSTM entraîné
    tokenizer -- le tokenizer utilisé pour convertir les noms de produits
    input_sequence -- la séquence d'entrée des produits
    num_predictions -- le nombre de prédictions à générer

    Retourne:
    Une liste des prochains produits les plus probables
    """
    for _ in range(num_predictions):
        # Convertir la séquence d'entrée en indices
        encoded_sequence = tokenizer.texts_to_sequences([input_sequence])[0]
        # Ajouter un padding pour correspondre à la longueur maximale de séquence
        padded_sequence = pad_sequences([encoded_sequence], maxlen=max_sequence_length, padding='pre')

        # Prédire le produit suivant
        predicted_index = model.predict(padded_sequence, verbose=0)
        predicted_index = np.argmax(predicted_index, axis=1)[0]

        # Convertir l'indice prédit en produit
        predicted_product = tokenizer.index_word[predicted_index]
        
        # Ajouter le produit prédit à la séquence d'entrée
        input_sequence += ' ' + predicted_product

    return input_sequence.split()[-num_predictions:]

# 13. Tester les prédictions
example_sequence = "product_1 product_2 product_3"
predicted_products = predict_next_products(model, tokenizer, example_sequence, num_predictions=3)
print(f"Produits prédits : {predicted_products}")

# Étape 7 : Questions de réflexion
# 14. Améliorations possibles
# Mettez en pratique les suggestions ci-dessous pour améliorer et évaluer le modèle

# Quelles approches pourriez-vous utiliser pour améliorer les performances du modèle ?
print("Suggestions d'améliorations pour les performances du modèle:")
print("- Augmenter la taille du dataset pour capturer plus de variations.")
print("- Utiliser des techniques de régularisation telles que la L2 regularization ou le dropout pour éviter le surapprentissage.")
print("- Expérimenter avec des architectures de modèles différentes, comme les GRU ou Transformer.")
print("- Fine-tuning des hyperparamètres tels que le taux d'apprentissage, la taille des couches, et le nombre de neurones.")

# Proposez des métriques d’évaluation adaptées pour ce type de système de recommandation
print("\nMétriques d'évaluation adaptées pour ce type de système de recommandation:")
print("- Précision@k : Pourcentage de fois où le produit correct est parmi les k premiers produits recommandés.")
print("- Recall@k : Pourcentage de fois où tous les produits corrects sont parmi les k premiers produits recommandés.")
print("- F1-Score : Harmonie entre la précision et le rappel pour évaluer le modèle.")
print("- AUC-ROC : Mesure la performance du modèle de classification à différents seuils de classification.")
print("- NDCG (Normalized Discounted Cumulative Gain) : Mesure l'efficacité des recommandations en prenant en compte la position des produits recommandés.")


Aperçu des colonnes :
Index(['customer_id', 'customer_first_name', 'customer_last_name',
       'category_name', 'product_name', 'customer_segment', 'customer_city',
       'customer_state', 'customer_country', 'customer_region',
       'delivery_status', 'order_date', 'order_id', 'ship_date',
       'shipping_type', 'days_for_shipment_scheduled',
       'days_for_shipment_real', 'order_item_discount', 'sales_per_order',
       'order_quantity', 'profit_per_order'],
      dtype='object')

Premières lignes du dataset :
  customer_id customer_first_name customer_last_name    category_name  \
0  C_ID_45866                Mary             Fuller  Office Supplies   
1  C_ID_44932                Alan            Edelman  Office Supplies   
2  C_ID_70880                Mary             Gayman  Office Supplies   
3  C_ID_33157             Raymond              Eason  Office Supplies   
4  C_ID_58303                Mary           Gonzalez        Furniture   

                           product_na

C:\Users\cated\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 86s 93ms/step - accuracy: 0.0161 - loss: 6.9091 - val_accuracy: 0.0496 - val_loss: 6.0094
Epoch 2/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 83s 98ms/step - accuracy: 0.0777 - loss: 5.6432 - val_accuracy: 0.1732 - val_loss: 4.8246
Epoch 3/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 80s 94ms/step - accuracy: 0.1998 - loss: 4.4608 - val_accuracy: 0.3230 - val_loss: 3.7451
Epoch 4/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 83s 98ms/step - accuracy: 0.3445 - loss: 3.4161 - val_accuracy: 0.4618 - val_loss: 2.9491
Epoch 5/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 80s 95ms/step - accuracy: 0.4539 - loss: 2.6736 - val_accuracy: 0.5489 - val_loss: 2.4398
Epoch 6/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 93s 107ms/step - accuracy: 0.5456 - loss: 2.1435 - val_accuracy: 0.6213 - val_loss: 2.0864
Epoch 7/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 127s 89ms/step - accuracy: 0.6069 - loss: 1.8002 - val_accuracy: 0.6630 - val_loss: 1.8558
Epoch 8/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 89s 105ms/step - accuracy: 0.6462 - loss: 1.5498

# Rapport Technique : Système de Recommandation de Produits

## Introduction
Ce rapport décrit le processus de développement d'un système de recommandation de produits utilisant un modèle GRU (Gated Recurrent Unit). Le code est divisé en plusieurs étapes allant de l'importation des bibliothèques nécessaires à la création et l'évaluation du modèle. 

## Importation des bibliothèques
Le code commence par l'importation des bibliothèques nécessaires pour la manipulation des données, la création du modèle et l'évaluation des performances.

## Chargement et exploration des données
Les données sont chargées à partir d'un fichier CSV. Les premières étapes consistent à afficher les colonnes du dataset ainsi que les premières lignes pour obtenir une vue d'ensemble des données.

## Prétraitement des données
Les données sont prétraitées pour ne conserver que les colonnes `customer_id`, `product_name` et `order_date`. Une réduction de la taille du dataset est effectuée en sélectionnant aléatoirement 20% des données. Les dates sont ensuite converties au format datetime et les enregistrements avec des dates invalides sont supprimés. Les données sont triées par `customer_id` et `order_date`, puis les séquences de produits par client sont agrégées.

## Tokenisation et création des paires d'entrée-sortie
Les séquences de produits sont tokenisées et transformées en indices numériques. Des paires d'entrée-sortie sont ensuite créées pour entraîner le modèle.

## Création et entraînement du modèle GRU
Le modèle GRU est défini avec une couche d'embedding, deux couches GRU, des couches de dropout pour éviter le surapprentissage et des couches denses pour la classification. Le modèle est compilé avec l'optimiseur Adam et la fonction de perte `sparse_categorical_crossentropy`, puis entraîné sur les données.

## Prédiction
Une fonction de prédiction est définie pour prédire les prochains produits les plus probables en utilisant le modèle entraîné. La fonction prend une séquence d'entrée et génère le nombre de prédictions souhaité.

## Questions de réflexion et améliorations possibles
Quelques suggestions pour améliorer les performances du modèle sont proposées, telles que l'augmentation de la taille du dataset, l'utilisation de techniques de régularisation, l'expérimentation avec différentes architectures de modèles et le réglage fin des hyperparamètres.

Des métriques d'évaluation adaptées pour ce type de système de recommandation sont également proposées, notamment la Précision@k, le Recall@k, le F1-Score, l'AUC-ROC et le NDCG.

## Conclusion
Ce rapport présente une méthode de développement d'un système de recommandation de produits basé sur un modèle GRU. Les étapes vont du chargement et du prétraitement des données à la création, l'entraînement et l'évaluation du modèle. Des suggestions d'améliorations et des métriques d'évaluation sont également fournies pour optimiser les performances du système.

